## Подготовка

### Импорт библиотек

Анализ данных:

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model

ModuleNotFoundError: No module named 'numpy'

Визуализация данных:

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib'

Виджеты для JupyterNotebook:

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

### Глобальные переменные

Датасет:

In [4]:
df = pd.read_csv("russian_demography.csv", delimiter=",")

NameError: name 'pd' is not defined

Обновляемый виджет-дисплей:

In [5]:
out = widgets.Output()

## Вспомогательные функции

### `gen_dfs(data, region, metric, year_to_predict)`

Генерирует наборы данных для визуализации.

__Принимает:__
1. Загруженный выше `df`
2. Регион
3. Требуемую метрику
4. Год, до которого строить прогноз.

__Возвращает наборы данных:__
1. С измерениями из `df`
2. С прогнозами по `df`
3. Объединенный набор данных

In [6]:
def gen_dfs(data, region, metric, year_to_predict):
    sr_df = df.loc[df["region"] == region].dropna()
    x = np.array(sr_df["year"]).reshape(-1, 1)
    y = np.array(sr_df[metric]).reshape(-1, 1)

    regr = linear_model.LinearRegression()
    regr.fit(X=x, y=y)
    
    x_future = np.array([year + 1 for year in range(sr_df["year"].max(), year_to_predict)]).reshape(-1, 1)
    y_future = regr.predict(x_future)

    c_df = pd.DataFrame({"year": x.flatten(), metric: y.flatten()})
    p_df = pd.DataFrame({"year": x_future.flatten(), metric: y_future.flatten()})
    u_df = pd.DataFrame({"year": np.append(x, x_future), metric: np.append(y, y_future)})

    return c_df, p_df, u_df    

### `draw(c_df, p_df, u_df, region)`

Визуализирует текущие измерения и прогноз на одном графике.

__Принимает в себя:__
1. Набор данных с измерениями из `df`
2. Набор данных с прогнозами по `df`
3. Объединенный набор данных

In [7]:
def draw(c_df, p_df, u_df, region):
    with out:
        clear_output(True)
        fig, ax = plt.subplots(figsize=(10, 6))
        fig.suptitle(f"Visualization of the forecast for {region}")
        ax.set(xticks=u_df["year"].values)
        
        sns.scatterplot(ax=ax, data=c_df, x=c_df.columns[0], y=c_df.columns[1], color="blue")
        sns.scatterplot(ax=ax, data=p_df, x=p_df.columns[0], y=p_df.columns[1], color="red")
        sns.lineplot(ax=ax, data=u_df, x=u_df.columns[0], y=u_df.columns[1], color="gray", errorbar=None)

        plt.xticks(rotation=75)
        plt.grid()
        plt.show()

## Виджеты

In [8]:
wRegion = widgets.Dropdown(
    options=df["region"].unique(),
    description="Регион: ",
)

wMetric = widgets.Dropdown(
    options=df.columns[2:],
    description="Метрика: ",
)

wYears = widgets.IntSlider(
    min=df["year"].max() + 1,
    max=df["year"].max() + 20,
    description="Прогноз до:",
    style={"description_width": "initial"}
)

def on_btn_click(b):
    c_df, p_df, u_df = gen_dfs(data=df,region=wRegion.value,
                               metric=wMetric.value,
                               year_to_predict=wYears.value)
    draw(c_df, p_df, u_df, wRegion.value)
    
wBtnDraw = widgets.Button(
    description="График"
)

wBtnDraw.on_click(on_btn_click)

hBoxUp = widgets.HBox([wRegion, wMetric])
hBoxMid = widgets.HBox([wYears])
hBoxDown = widgets.HBox([wBtnDraw])
vBox = widgets.VBox([hBoxUp, hBoxMid, hBoxDown])

display(vBox)
out

Output()